Some of this code is Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
# Uncomment for just the 108 epoch dataset (~500mb)
# !curl -O https://storage.googleapis.com/nasbench/nasbench_only108.tfrecord
# Uncomment for the full dataset (~2gb)
# !curl -O https://storage.googleapis.com/nasbench/nasbench_full.tfrecord

In [ ]:
import math

from trials.Constants import *
from trials.Evolution import *
from trials.ModelSpec import *
from trials.Search import *
from trials.Selection import *
from trials.Utilities import *

from IPython.core.display import clear_output

In [3]:
def run_random_search(k: int, max_time: float):
    """
    Runs a simulated random search for a fixed amount of time. Actual time taken will be
    relatively quick, only the simulated time is considered. Will return all evaluated models,
    along with the k best models found.
    :param k: Number of best models to return
    :param max_time_budget: The amount of simulated time to expend.
    :return: All models evaluated, along with the best k models.
    """
    # resetting seeds at the top of the search functions is required for reproducibility
    np.random.seed(RNG_SEED)
    random.seed(RNG_SEED)

    # nasbench tracks time taken with each query, effectively simulating full
    # training, as we can query how long into the training we would be if we
    # were training ourselves. we reset the counters at the top of each experiment
    nasbench.reset_budget_counters()

    best: List[SpecWrapper] = []
    models: List[SpecWrapper] = []
    evaluated: Set[str] = set()
    collisions: int = 0

    time_spent, _ = nasbench.get_budget_counters()
    while time_spent < max_time:
        spec = random_spec()
        spec_hash = spec.hash_spec(nasbench.config["available_ops"])

        if spec_hash in evaluated:
            collisions += 1
            continue
        else:
            evaluated.add(spec_hash)

        models.append(spec)
        best = sel_best(models, k)
        time_spent, _ = nasbench.get_budget_counters()
        clear_output(wait=True)
        print(f"{time_spent/1000:0.2f}/{max_time/1000:0.0f}k ({(time_spent/max_time)*100:0.2f}%) seconds simulated")

    print(f"{len(evaluated)} unique models during random search.")
    print(f"{collisions} collisions during random search.")

    return models, best

In [4]:
from typing import Set
from typing import Callable


def get_time_taken():
    time_spent, _ = nasbench.get_budget_counters()
    return time_spent


In [7]:

# TODO: Mutation function creators, define experiment structure

def search(
        max_time: float,
        num_best: int,
        num_epochs: int,
        initial_population: List[SpecWrapper],
        mut_fn: Callable[[List[SpecWrapper]], List[SpecWrapper]],
        sel_fn: Callable[[List[SpecWrapper]], List[SpecWrapper]],
        drp_fn: Callable[[List[SpecWrapper]], List[SpecWrapper]]
    ) -> [List[List[SpecWrapper], List[SpecWrapper], Set[str]]]:

    sel_best = sel_best_fn(num_best)

    def run_epoch(epoch_num: int, population: List[SpecWrapper]):
        # initialize our population list, update budget counters
        population = [get_spec(ind.get_hash()) for ind in population]
        [ind.get_data() for ind in population]

        # desired size of the population
        p_size = len(population)

        reset_trial_stats(RNG_SEED + epoch_num)

        # list of hashes that we have previously evaluated which can be skipped in the future
        done: Set[str] = set()

        # update done, adding any new hashes to our set
        def update_done(items: List[SpecWrapper]):
            done.update(map(lambda x: x.get_hash(), items))

        update_done(population)

        # running cumulative time total of all epochs
        cur_time: float = 0

        while cur_time < max_time:
            [ind.get_data() for ind in population]

            # drop some candidates base on the specified function
            population = drp_fn(population)
            assert len(population) > 0

            # number of new specs to generate through mutation or crossover
            num_new = p_size - len(population)
            new_specs: List[SpecWrapper] = []
            while len(new_specs) < num_new:
                # select some candidates to mutate
                candidates = sel_fn(population)
                # mutate the candidates with the fn
                candidates = mut_fn(candidates)

                # only "evaluate" candidates who were not evaluate
                # so if we hit a duplicate candidate, we should not
                # add more training time, as it is wasteful
                cand_hashes = [cand.get_hash() for cand in candidates]
                cand_hashes = [hsh for hsh in cand_hashes if hsh not in done]

                # remove dupe hashes
                candidates = [get_spec(hsh) for hsh in cand_hashes]

                # update the list of new specs
                new_specs = [*new_specs, *candidates]

            population = [*population, *new_specs][:p_size]
            [ind.get_data() for ind in population]

            cur_time = get_time_taken()
            print(f"{cur_time/1000:0.2f}/{max_time/1000:0.0f}k ({(cur_time/max_time)*100:0.2f}%) seconds simulated")

        best = sel_best(population)
        best.sort()

        abs_best = best[-1].get_data()
        print(f'Best in trial -- Test: {abs_best.test_accuracy:0.7f}, Valid: {abs_best.valid_accuracy:0.7f}')

        return population, best, done

    results: [List[List[SpecWrapper], List[SpecWrapper], Set[str]]] = []
    for epoch_num in range(num_epochs):
        population, best, done = run_epoch(epoch_num, initial_population)
        results.append([population, best, done])

    return results



TypeError: Too many parameters for typing.List; actual 3, expected 1